In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os
from pathlib import Path

import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_corr_triangle
from core.s3 import S3AssetManager

In [2]:
notebook_name = "okuo_segundo_ensayo_acidez_h2o"
s3 = S3AssetManager(notebook_name=notebook_name)
PALETTE =  [ "#1c8074","#1a494c", "#94af92", "#666666", "#f9ee77", "#f5ad68", "#c76931"]

In [3]:
def filter_post_dose(df: pd.DataFrame) -> pd.DataFrame:
    """Quita la lectura inicial si pH > 6."""
    mask = ~((df["elapsed_h"] == 0) & (df["pH"] > 9))
    return df.loc[mask].copy() # & (df["elapsed_h"]>0)

def add_elapsed_hours(df: pd.DataFrame) -> pd.DataFrame:
    """Añade columna 'elapsed_h' por producto."""
    first = df.groupby("producto")["tiempo"].min()
    df["elapsed_h"] = df.apply(
        lambda r: (r["tiempo"] - first[r["producto"]]).total_seconds() / 3600,
        axis=1,
    )
    return df



def stability_table(df: pd.DataFrame,) -> pd.DataFrame:
    """Devuelve CV, SD y rango de pH por producto."""
    stab = (
        df.groupby("producto")
          .agg(
              n   = ("pH",  "count"),
              # ---- pH ----
              pH_mean   = ("pH",  "median"),
              pH_sd     = ("pH",  "std"),
              pH_range  = ("pH",  lambda x: x.max() - x.min()),
              pH_cv_pct = ("pH",  lambda x: x.std() / x.mean() * 100),

              # ---- ORP ----
              ORP_mean   = ("ORP", "median"),
              ORP_sd     = ("ORP", "std"),
              ORP_range  = ("ORP", lambda x: x.max() - x.min()),
              ORP_cv_pct = ("ORP", lambda x: x.std() / x.mean() * 100),
          )
          .round(3)
          .sort_values("pH_sd")        # ordena por menor variabilidad de pH
    )
    return stab


# ------------------------------------------------------------------
# 1) Curva temporal (pH, ORP, etc.)
# ------------------------------------------------------------------
def plot_metric_curve(
    df: pd.DataFrame,
    metric: str,
    x: str = "elapsed_h",
    color_col: str = "producto",
    palette: list[str] = PALETTE,
    title: str | None = None,
    yaxis_title: str | None = None,
    save_html: str | Path | None = None,
    annotate_initial: bool = True,
) -> go.Figure:

    # Mapa producto → color
    prods      = df[color_col].unique()
    color_map  = {p: palette[i % len(palette)] for i, p in enumerate(prods)}

    # Línea principal
    fig = px.line(
        df, x=x, y=metric, color=color_col,
        color_discrete_map=color_map, markers=True,
        title=title or f"Evolución de {metric.upper()}",
    )
    fig.update_layout(
    title=dict(font=dict(color="black"), x=0.5)  # x=0.5 lo centra
    )

    # Ejes y estética en negro
    fig.update_layout(
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        legend_title="",
        legend_font_color="black",
        xaxis=dict(
            title="Horas transcurridas",
            title_font_color="black",
            tickfont_color="black",
            linecolor="black",
            mirror=True, showline=True,             # marco
            gridcolor="rgba(0,0,0,0.1)",
        ),
        yaxis=dict(
            title=yaxis_title or metric.upper(),
            title_font_color="black",
            tickfont_color="black",
            linecolor="black",
            mirror=True, showline=True,
            gridcolor="rgba(0,0,0,0.1)",
        ),
    )

    # Marco negro (shape) – opcional, realza el borde
    fig.add_shape(type="rect",
                  xref="paper", yref="paper",
                  x0=0, y0=0, x1=1, y1=1,
                  line=dict(color="black", width=1))

    # Flecha en t=0
    if annotate_initial:
        ini_vals = df[df[x] == 0][metric]
        if not ini_vals.empty:
            fig.add_annotation(
                x=0, y=ini_vals.max(),
                text="condición inicial",
                showarrow=True, arrowhead=3,
                arrowwidth=1.5, arrowcolor="#C9C9C9",
                ax=40, ay=-40,
                font=dict(color="#666666"),
                #bgcolor="rgba(0,0,0,0)",
                bordercolor="#666666",
            )

    if save_html:
        save_path = Path(save_html)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        fig.write_html(save_path, include_plotlyjs="cdn")
        print(f"[INFO] Curva guardada en {save_path.resolve()}")

    return fig

# ------------------------------------------------------------------
# 2) Barra de estabilidad (SD pH) con CV %
# ------------------------------------------------------------------
def plot_ph_stability_bar(
    stab: pd.DataFrame,
    palette: list[str] = PALETTE,
    save_html: str | Path | None = None,
) -> go.Figure:

    stab = stab.copy()
    stab["producto"] = stab.index
    stab = stab.sort_values("pH_sd")  # ordenar de más estable a menos

    fig = px.bar(
        stab,
        x="producto", y="pH_sd",
        color="producto",
        color_discrete_sequence=palette * 3,
        #text=stab["pH_cv_pct"].map(lambda v: f"{v:.2f}%"),
        title="Estabilidad del pH con los productos",
    )
    fig.update_layout(
    title=dict(font=dict(color="black"), x=0.5)  # x=0.5 lo centra
    )

    fig.update_traces(
        textposition="outside",
        textfont_color="black",
    )

    fig.update_layout(
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        showlegend=False,
        xaxis=dict(
            title="Producto",
            title_font_color="black",
            tickfont_color="black",
            linecolor="black",
            mirror=True, showline=True,
        ),
        yaxis=dict(
            title="Desviación estándar de pH",
            title_font_color="black",
            tickfont_color="black",
            linecolor="black",
            mirror=True, showline=True,
            gridcolor="rgba(0,0,0,0.1)",
        ),
    )

    # Marco negro
    fig.add_shape(type="rect",
                  xref="paper", yref="paper",
                  x0=0, y0=0, x1=1, y1=1,
                  line=dict(color="black", width=1))

    if save_html:
        p = Path(save_html); p.parent.mkdir(exist_ok=True, parents=True)
        fig.write_html(p, include_plotlyjs="cdn")
        print(f"[INFO] Barra guardada en {p.resolve()}")

    return fig


from pathlib import Path
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

def plot_stability_bar(
    df: pd.DataFrame,
    y_col: str,
    palette: list[str] = None,
    save_html: str | Path | None = None,
    y_label: str | None = None,
    title: str = "Estabilidad de la variable",
) -> go.Figure:

    if palette is None:
        palette = px.colors.qualitative.Plotly

    df = df.copy()
    df["producto"] = df.index
    df = df.sort_values(y_col)  # ordenar de menor a mayor valor

    fig = px.bar(
        df,
        x="producto",
        y=y_col,
        color="producto",
        color_discrete_sequence=palette * 3,
        title=title,
    )

    fig.update_traces(
        textposition="outside",
        textfont_color="black",
    )

    fig.update_layout(
        title=dict(font=dict(color="black"), x=0.5),
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        showlegend=False,
        xaxis=dict(
            title="Producto",
            title_font_color="black",
            tickfont_color="black",
            linecolor="black",
            mirror=True,
            showline=True,
        ),
        yaxis=dict(
            title=y_label or y_col,
            title_font_color="black",
            tickfont_color="black",
            linecolor="black",
            mirror=True,
            showline=True,
            gridcolor="rgba(0,0,0,0.1)",
        ),
    )

    # Marco negro
    fig.add_shape(
        type="rect",
        xref="paper", yref="paper",
        x0=0, y0=0, x1=1, y1=1,
        line=dict(color="black", width=1),
    )

    if save_html:
        p = Path(save_html)
        p.parent.mkdir(exist_ok=True, parents=True)
        fig.write_html(p, include_plotlyjs="cdn")
        print(f"[INFO] Gráfico guardado en {p.resolve()}")

    return fig


# ------------------------------------------------------------------
# 3) Dispersión ORP vs pH
# ------------------------------------------------------------------
def plot_orp_vs_ph(
    df: pd.DataFrame,
    ph_col: str = "pH",
    orp_col: str = "ORP",
    color_col: str = "producto",
    palette: list[str] = PALETTE,
    save_html: str | Path | None = None,
) -> go.Figure:

    prods     = df[color_col].unique()
    color_map = {p: palette[i % len(palette)] for i, p in enumerate(prods)}

    fig = px.scatter(
        df, x=ph_col, y=orp_col,
        color=color_col,
        color_discrete_map=color_map,
        opacity=1,
        title="Relación ORP vs pH",
    )
    fig.update_layout(
    title=dict(font=dict(color="black"), x=0.5)  # x=0.5 lo centra
    )

    fig.update_layout(
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        legend_title="",
        legend_font_color="black",
        xaxis=dict(
            title="pH",
            title_font_color="black",
            tickfont_color="black",
            linecolor="black",
            mirror=True, showline=True,
            autorange="reversed",
            gridcolor="rgba(0,0,0,0.1)",
        ),
        yaxis=dict(
            title="ORP (mV)",
            title_font_color="black",
            tickfont_color="black",
            linecolor="black",
            mirror=True, showline=True,
            gridcolor="rgba(0,0,0,0.1)",
        ),
    )

    # Marco negro
    fig.add_shape(type="rect",
                  xref="paper", yref="paper",
                  x0=0, y0=0, x1=1, y1=1,
                  line=dict(color="black", width=1))

    # Anotación en pH 6.5, ORP 650
    fig.add_annotation(
        x=6.5, y=650,
        text="condición inicial",
        showarrow=True,
        arrowhead=3, arrowsize=1, arrowwidth=1.5,
        arrowcolor="#C9C9C9",
        ax=40, ay=-40,
        font=dict(color="#666666"),
        #bgcolor="rgba(255,255,255,0.8)",
        bordercolor="#666666",
    )

    if save_html:
        p = Path(save_html); p.parent.mkdir(exist_ok=True, parents=True)
        fig.write_html(p, include_plotlyjs="cdn")
        print(f"[INFO] Dispersión guardada en {p.resolve()}")

    return fig



from plotly.subplots import make_subplots
import plotly.graph_objects as go

def plot_metric_subplot(
    df0: pd.DataFrame,
    df1: pd.DataFrame,
    metric: str,
    x: str = "elapsed_h",
    color_col: str = "producto",
    palette: list[str] = PALETTE,
    title0: str = "Evolución del pH iniciando en condiciones normales",
    title1: str = "Evolución del pH iniciando en condiciones de pH bajo",
    save_html: str | Path | None = None,
) -> go.Figure:

    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        subplot_titles=[title0, title1]
    )

    def add_trace_group(fig, df, row, title):
        prods = df[color_col].unique()
        color_map = {p: palette[i % len(palette)] for i, p in enumerate(prods)}

        for prod in prods:
            df_prod = df[df[color_col] == prod]
            fig.add_trace(
                go.Scatter(
                    x=df_prod[x],
                    y=df_prod[metric],
                    mode="lines+markers",
                    name=prod,
                    legendgroup=prod,
                    marker=dict(color=color_map[prod]),
                    line=dict(color=color_map[prod]),
                ),
                row=row, col=1
            )

    # Añadir ambos conjuntos de datos
    add_trace_group(fig, df0, row=1, title=title0)
    add_trace_group(fig, df1, row=2, title=title1)

    fig.update_layout(
        height=700,
        showlegend=True,
        title_text="Comparación de evolución del pH en dos condiciones",
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        legend_title_text="",
        xaxis=dict(
            title="Horas transcurridas",
            titlefont=dict(color="black"),
            tickfont=dict(color="black"),
            linecolor="black",
            mirror=True,
            showline=True,
            gridcolor="rgba(0,0,0,0.1)",
        ),
        yaxis=dict(
            title="pH",
            titlefont=dict(color="black"),
            tickfont=dict(color="black"),
            linecolor="black",
            mirror=True,
            showline=True,
            gridcolor="rgba(0,0,0,0.1)",
        )
    )

    # Guardar si se requiere
    if save_html:
        save_path = Path(save_html)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        fig.write_html(save_path, include_plotlyjs="cdn")
        print(f"[INFO] Subplot guardado en {save_path.resolve()}")

    return fig



from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pathlib import Path

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pathlib import Path

def plot_metric_subplot_1x2(
    df0: pd.DataFrame,
    df1: pd.DataFrame,
    metric: str,
    x: str = "elapsed_h",
    color_col: str = "producto",
    palette: list[str] = PALETTE,
    title0: str = "Condición inicial de pH normal",
    title1: str = "Condición inicial de pH bajo",
    save_html: str | Path | None = None,
) -> go.Figure:

    # Crear subplot 1x2
    fig = make_subplots(
        rows=1, cols=2,
        shared_yaxes=True,
        horizontal_spacing=0.01,
        subplot_titles=[title0, title1]
    )

    def add_traces(df, col):
        prods = df[color_col].unique()
        color_map = {p: palette[i % len(palette)] for i, p in enumerate(prods)}
        for prod in prods:
            df_prod = df[df[color_col] == prod]
            fig.add_trace(
                go.Scatter(
                    x=df_prod[x],
                    y=df_prod[metric],
                    mode="lines+markers",
                    name=prod,
                    legendgroup=prod,
                    showlegend=(col == 1),
                    line=dict(color=color_map[prod], width=3),
                    marker=dict(color=color_map[prod]),
                ),
                row=1, col=col
            )

    add_traces(df0, col=1)
    add_traces(df1, col=2)

    # Estilo general
    fig.update_layout(
        height=500,
        width=1000,
        title_text="Evolución del pH en dos condiciones iniciales",
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        title=dict(x=0.5, font=dict(color="black")),
        legend_title_text="",
        font=dict(color="black"),
    )

    # Ejes X
    fig.update_xaxes(
        title_text="Horas transcurridas",
        title_font=dict(color="black"),
        tickfont=dict(color="black"),
        linecolor="black",
        mirror=True, showline=True,
        gridcolor="rgba(0,0,0,0.1)",
    )

    # Ejes Y
    fig.update_yaxes(
        title_text="pH",
        title_font=dict(color="black"),
        tickfont=dict(color="black"),
        linecolor="black",
        mirror=True, showline=True,
        gridcolor="rgba(0,0,0,0.1)",
    )

    # Guardar como HTML si se especifica
    if save_html:
        save_path = Path(save_html)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        fig.write_html(save_path, include_plotlyjs="cdn")
        print(f"[INFO] Subplot guardado en {save_path.resolve()}")

    return fig

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from pathlib import Path
import pandas as pd
from typing import Union

def plot_metric_subplot_1x3(
    df0: pd.DataFrame,
    df1: pd.DataFrame,
    df2: pd.DataFrame,
    metric: str,
    x: str = "elapsed_h",
    title_text:str = "",
    y_title:str = "",
    color_col: str = "producto",
    palette: list[str] = PALETTE,
    title0: str = "Condición A",
    title1: str = "Condición B",
    title2: str = "Condición C",
    save_html: Union[str, Path, None] = None,
) -> go.Figure:

    # Crear subplot 1x3
    fig = make_subplots(
        rows=1, cols=3,
        shared_yaxes=True,
        horizontal_spacing=0.03,
        subplot_titles=[title0, title1, title2]
    )

    def add_traces(df, col):
        prods = df[color_col].unique()
        color_map = {p: palette[i % len(palette)] for i, p in enumerate(prods)}
        for prod in prods:
            df_prod = df[df[color_col] == prod]
            fig.add_trace(
                go.Scatter(
                    x=df_prod[x],
                    y=df_prod[metric],
                    mode="lines+markers",
                    name=prod,
                    legendgroup=prod,
                    showlegend=(col == 1),
                    line=dict(color=color_map[prod], width=3),
                    marker=dict(color=color_map[prod]),
                ),
                row=1, col=col
            )

    # Agregar datos a cada columna
    add_traces(df0, col=1)
    add_traces(df1, col=2)
    add_traces(df2, col=3)

    # Estilo general
    fig.update_layout(
        height=500,
        width=1300,
        title_text=title_text,
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        title=dict(x=0.5, font=dict(color="black")),
        legend_title_text="",
        font=dict(color="black"),
    )

    # Ejes X
    fig.update_xaxes(
        title_text="Horas transcurridas",
        title_font=dict(color="black"),
        tickfont=dict(color="black"),
        linecolor="black",
        mirror=True, showline=True,
        gridcolor="rgba(0,0,0,0.1)",
    )

    # Ejes Y
    fig.update_yaxes(
        title_text=y_title,
        title_font=dict(color="black"),
        tickfont=dict(color="black"),
        linecolor="black",
        mirror=True, showline=True,
        gridcolor="rgba(0,0,0,0.1)",
    )

    # Guardar como HTML si se requiere
    if save_html:
        save_path = Path(save_html)
        save_path.parent.mkdir(parents=True, exist_ok=True)
        fig.write_html(save_path, include_plotlyjs="cdn")
        print(f"[INFO] Subplot guardado en {save_path.resolve()}")

    return fig


import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from pathlib import Path

def plot_scatter_by_group(
    df: pd.DataFrame,
    x_col: str,
    y_col: str,
    color_col: str = "grupo",
    palette: list[str] = None,
    annotation: tuple[float, float, str] = None,  # (x, y, texto)
    save_html: str | Path | None = None,
    title: str = None,
) -> go.Figure:
    """
    Crea un scatter plot personalizado entre dos variables, con agrupación por color.

    Parámetros:
    - df: DataFrame con los datos.
    - x_col: Nombre de la columna del eje X.
    - y_col: Nombre de la columna del eje Y.
    - color_col: Columna para diferenciar grupos con color.
    - palette: Lista de colores para los grupos (se repetirá si hay más grupos).
    - annotation: Tupla opcional (x, y, texto) para agregar una anotación.
    - save_html: Ruta opcional para guardar el gráfico como archivo HTML.
    - title: Título del gráfico (opcional).

    Retorna:
    - fig: Objeto Plotly Figure.
    """
    if palette is None:
        palette = px.colors.qualitative.Pastel

    grupos = df[color_col].unique()
    color_map = {g: palette[i % len(palette)] for i, g in enumerate(grupos)}

    fig = px.scatter(
        df, x=x_col, y=y_col,
        color=color_col,
        color_discrete_map=color_map,
        opacity=1,
        title=title or f"Relación entre {y_col} y {x_col}"
    )

    fig.update_layout(
        title=dict(x=0.5, font=dict(color="black")),
        plot_bgcolor="rgba(0,0,0,0)",
        paper_bgcolor="rgba(0,0,0,0)",
        legend_title="",
        legend_font_color="black",
        xaxis=dict(
            title=x_col,
            title_font_color="black",
            tickfont_color="black",
            linecolor="black",
            mirror=True,
            showline=True,
            gridcolor="rgba(0,0,0,0.1)",
        ),
        yaxis=dict(
            title=y_col,
            title_font_color="black",
            tickfont_color="black",
            linecolor="black",
            mirror=True,
            showline=True,
            gridcolor="rgba(0,0,0,0.1)",
        ),
    )

    # Marco negro
    fig.add_shape(
        type="rect",
        xref="paper", yref="paper",
        x0=0, y0=0, x1=1, y1=1,
        line=dict(color="black", width=1)
    )

    # Anotación opcional
    if annotation:
        x, y, texto = annotation
        fig.add_annotation(
            x=x, y=y,
            text=texto,
            showarrow=True,
            arrowhead=3,
            arrowsize=1,
            arrowwidth=1.5,
            arrowcolor="#C9C9C9",
            ax=40, ay=-40,
            font=dict(color="#666666"),
            bordercolor="#666666",
        )

    if save_html:
        p = Path(save_html)
        p.parent.mkdir(parents=True, exist_ok=True)
        fig.write_html(p, include_plotlyjs="cdn")
        print(f"[INFO] Dispersión guardada en {p.resolve()}")

    return fig



In [ ]:
raw = s3.read_excel(
    "raw/ensayos_acidez/2° EVALUACION ACIDIFICANTES CONTROLADO CLORO.xlsx",sheet_name="ENSAYO", skiprows=18
)
df = raw.loc[:, ['producto', 'pH', 'ORP', 'FCL', 'TCL',  'dia',  'hora', 'ref']]
df = df[df["producto"].notnull()].copy()
df["timestamp"] = pd.to_datetime(df["dia"].astype(str) + " " + df["hora"].astype(str))

In [5]:
df.head()

,producto,pH,ORP,FCL,TCL,dia,hora,ref,timestamp
0,Citroquim Aqua,9.21,148,NaN,NaN,2025-07-22,12:00:00,0,2025-07-22 12:00:00
1,Citroquim Aqua,9.21,148,NaN,NaN,2025-07-22,12:00:00,1,2025-07-22 12:00:00
2,Citroquim Aqua,9.21,148,NaN,NaN,2025-07-22,12:00:00,2,2025-07-22 12:00:00
3,Selko pH,9.18,317,NaN,NaN,2025-07-22,03:00:00,0,2025-07-22 03:00:00
4,Selko pH,9.18,317,NaN,NaN,2025-07-22,03:00:00,1,2025-07-22 03:00:00


In [6]:
first = df.groupby("producto")["timestamp"].min()
df["elapsed_h"] = df.apply(
        lambda r: (r["timestamp"] - first[r["producto"]]).total_seconds() / 3600,
        axis=1,
    )


In [8]:
df_post  = filter_post_dose(df)
df_post

,producto,pH,ORP,FCL,TCL,dia,hora,ref,timestamp,elapsed_h
14,Citroquim Aqua,5.72,670,3.26,3.27,2025-07-22,12:00:00,0,2025-07-22 12:00:00,0.000000
15,Citroquim Aqua,4.37,755,3.19,3.53,2025-07-22,12:00:00,1,2025-07-22 12:00:00,0.000000
16,Citroquim Aqua,3.28,775,2.23,2.58,2025-07-22,12:00:00,2,2025-07-22 12:00:00,0.000000
17,Selko pH,5.68,722,4.29,7.36,2025-07-22,03:00:00,0,2025-07-22 03:00:00,0.000000
18,Selko pH,4.19,784,3.21,4.30,2025-07-22,03:00:00,1,2025-07-22 03:00:00,0.000000
19,Selko pH,3.52,400,1.22,2.87,2025-07-22,03:00:00,2,2025-07-22 03:00:00,0.000000
20,CLAP,5.62,674,4.02,5.25,2025-07-22,04:10:00,0,2025-07-22 04:10:00,0.000000
21,CLAP,4.26,717,3.67,4.34,2025-07-22,04:10:00,1,2025-07-22 04:10:00,0.000000
22,CLAP,3.48,775,3.37,4.26,2025-07-22,04:10:00,2,2025-07-22 04:10:00,0.000000
23,ACIDO ACETICO,5.25,703,3.56,4.31,2025-07-22,05:00:00,0,2025-07-22 05:00:00,0.000000


In [10]:
fig_ph0 = plot_metric_curve(
    df_post[df_post["ref"]==0],
    metric="pH",
    #save_html=f"{ROOT_IMAGEN}/ph_curva_0.html",
    annotate_initial=False,
    title="Evolución del PH iniciando en condiciones normales"
)
fig_ph0.show()
s3.save_plotly_html(fig_ph0, "ph_curva_0")

In [12]:
fig_ph1 = plot_metric_curve(
    df_post[df_post["ref"]==1],
    metric="pH",
    #save_html=f"{ROOT_IMAGEN}/ph_curva_1.html",
    annotate_initial=False,
    title="Evolución del PH iniciando en condiciones de pH bajo"
)
fig_ph1
s3.save_plotly_html(fig_ph1, "ph_curva_1")

In [13]:
fig_ph_subplot = plot_metric_subplot(
    df_post[df_post["ref"] == 0],
    df_post[df_post["ref"] == 1],
    metric="pH",
    #save_html=f"{ROOT_IMAGEN}/ph_subplot.html"
)
fig_ph_subplot.show()
s3.save_plotly_html(fig_ph_subplot, "ph_subplot")

In [14]:
fig_ph_subplot = plot_metric_subplot_1x2(
    df_post[df_post["ref"] == 0],
    df_post[df_post["ref"] == 1],
    metric="pH",
    #save_html=f"{ROOT_IMAGEN}/ph_subplot_1x2.html"
)
fig_ph_subplot.show()
s3.save_plotly_html(fig_ph_subplot, "ph_subplot_1x2.html")

In [16]:
fig_ph_subplot = plot_metric_subplot_1x3(
    df0=df_post[df_post["ref"] == 0],
    df1=df_post[df_post["ref"] == 1],
    df2=df_post[df_post["ref"] == 2],
    metric="pH",
    title_text= "",
    y_title="pH",
    title0="Condición inicial de pH normal",
    title1="Condición inicial de pH bajo",
    title2="Condición inicial de pH muy bajo",
    #save_html=f"{ROOT_IMAGEN}/ph_subplot_1x3.html"
)
fig_ph_subplot.show()
s3.save_plotly_html(fig_ph_subplot, "ph_subplot_1x3.html")

In [17]:
fig_ph_subplot = plot_metric_subplot_1x2(
    df_post[df_post["ref"] == 0],
    df_post[df_post["ref"] == 1],
    metric="ORP",

    #save_html=f"{ROOT_IMAGEN}/orp_subplot_1x2.html"
)
fig_ph_subplot.show()
s3.save_plotly_html(fig_ph_subplot, "orp_subplot_1x2.html")

In [18]:
fig_ph_subplot = plot_metric_subplot_1x3(
    df0=df_post[df_post["ref"] == 0],
    df1=df_post[df_post["ref"] == 1],
    df2=df_post[df_post["ref"] == 2],
    metric="ORP",
    title_text= "Evolución del ORP con condiciones inicial de pH",
    y_title="ORP",
    title0="Condición inicial de pH normal",
    title1="Condición inicial de pH bajo",
    title2="Condición inicial de pH muy bajo",
    #save_html=f"{ROOT_IMAGEN}/orp_subplot_1x3.html"
)
fig_ph_subplot.show()
s3.save_plotly_html(fig_ph_subplot, "orp_subplot_1x3.html")

In [19]:
fig_ph_subplot = plot_metric_subplot_1x3(
    df0=df_post[df_post["ref"] == 0],
    df1=df_post[df_post["ref"] == 1],
    df2=df_post[df_post["ref"] == 2],
    metric="TCL",
    title_text= "Evolución del ORP con condiciones inicial de pH",
    y_title="TCL",
    title0="Condición inicial de pH normal",
    title1="Condición inicial de pH bajo",
    title2="Condición inicial de pH muy bajo",
    #save_html=f"{ROOT_IMAGEN}/tcl_subplot_1x3.html"
)
fig_ph_subplot.show()
s3.save_plotly_html(fig_ph_subplot, "tcl_subplot_1x3.html")

In [21]:
fig_orp0 = plot_metric_curve(
    df_post[df_post["ref"]==2],
    metric="pH",
    #save_html=f"{ROOT_IMAGEN}/orp_curva_0.html"
)
fig_orp0.show()
s3.save_plotly_html(fig_orp0, "orp_curva_0.html")

In [22]:
fig_orp1 = plot_metric_curve(
    df_post[df_post["ref"]==0],
    metric="ORP",
    #save_html=f"{ROOT_IMAGEN}/orp_curva_1.html"
)
fig_orp1
s3.save_plotly_html(fig_orp1, "orp_curva_1.html")

In [23]:
fig_fcl0 = plot_metric_curve(
    df_post[df_post["ref"]==0],
    metric="FCL",
    #save_html=f"{ROOT_IMAGEN}/fcl0.html"
)
fig_fcl0
s3.save_plotly_html(fig_fcl0, "fcl0.html")

In [24]:
fig_fcl1 = plot_metric_curve(
    df_post[df_post["ref"]==1],
    metric="FCL",
    #save_html=f"{ROOT_IMAGEN}/fcl0.html"
)
fig_fcl1
s3.save_plotly_html(fig_fcl1, "fcl0.html")

In [25]:
df

,producto,pH,ORP,FCL,TCL,dia,hora,ref,timestamp,elapsed_h
0,Citroquim Aqua,9.21,148,NaN,NaN,2025-07-22,12:00:00,0,2025-07-22 12:00:00,0.00
1,Citroquim Aqua,9.21,148,NaN,NaN,2025-07-22,12:00:00,1,2025-07-22 12:00:00,0.00
2,Citroquim Aqua,9.21,148,NaN,NaN,2025-07-22,12:00:00,2,2025-07-22 12:00:00,0.00
3,Selko pH,9.18,317,NaN,NaN,2025-07-22,03:00:00,0,2025-07-22 03:00:00,0.00
4,Selko pH,9.18,317,NaN,NaN,2025-07-22,03:00:00,1,2025-07-22 03:00:00,0.00
...,...,...,...,...,...,...,...,...,...,...
65,ACIDO ACETICO,4.97,722,NaN,NaN,2025-07-23,15:00:00,0,2025-07-23 15:00:00,34.00
66,ACIDO ACETICO,4.21,824,NaN,NaN,2025-07-23,15:00:00,1,2025-07-23 15:00:00,34.00
67,ACIDO ACETICO,3.16,888,NaN,NaN,2025-07-23,15:00:00,2,2025-07-23 15:00:00,34.00
68,ACIDO FOSFORICO,4.39,793,NaN,NaN,2025-07-23,15:00:00,0,2025-07-23 15:00:00,33.75


In [26]:
fig_fcl2 = plot_metric_curve(
    df,
    metric="TCL",
    #save_html=f"{ROOT_IMAGEN}/fcl2.html"
)
fig_fcl2
s3.save_plotly_html(fig_fcl2, "fcl2.html")

In [28]:
stab0 = stability_table(df_post[(df["ref"]==0) & (df_post["elapsed_h"]>0)])
fig_stab0 = plot_ph_stability_bar(
    stab0,
    #save_html=f"{ROOT_IMAGEN}/ph_stability0.html"
)
fig_stab0
s3.save_plotly_html(fig_stab0, "ph_stability0.html")

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_58576/749576877.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [29]:
stab1 = stability_table(df_post[(df["ref"]==1) & (df_post["elapsed_h"]>0)])
fig_stab1 = plot_ph_stability_bar(
    stab1,
    #save_html=f"{ROOT_IMAGEN}/ph_stability1.html"
)
fig_stab1
s3.save_plotly_html(fig_stab1, "ph_stability1.html")

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_58576/356216572.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [30]:
stab2 = stability_table(df_post[(df["ref"]==2) & (df_post["elapsed_h"]>0)])
fig_stab2 = plot_ph_stability_bar(
    stab2,
    #save_html=f"{ROOT_IMAGEN}/ph_stability2.html"
)
fig_stab2
s3.save_plotly_html(fig_stab2, "ph_stability2.html")

/var/folders/1g/77kw2x4j5678s_87_sqc1fpc0000gp/T/ipykernel_58576/2831321.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [31]:

stab = stability_table((df_post[(df_post["elapsed_h"]>0)]))
fig_stab = plot_ph_stability_bar(
    stab,
    #save_html=f"{ROOT_IMAGEN}/ph_stability.html"
)
fig_stab
s3.save_plotly_html(fig_stab, "ph_stability.html")

In [139]:
stab

,n,pH_mean,pH_sd,pH_range,pH_cv_pct,ORP_mean,ORP_sd,ORP_range,ORP_cv_pct
producto,,,,,,,,,
ACIDO FOSFORICO,6,3.92,0.77,1.76,20.20,885.50,69.57,159,7.88
ACIDO ACETICO,9,4.21,0.80,2.15,19.00,824.00,69.52,186,8.49
CLAP,9,3.88,1.09,2.84,25.64,694.00,109.39,336,16.39
Citroquim Aqua,9,4.36,1.19,3.14,25.89,621.00,157.80,447,28.54
Selko pH,9,4.18,1.26,3.21,26.66,521.00,190.16,548,36.31


In [140]:
plot_stability_bar(
    stab0,
    y_col="pH_sd",
    y_label="Desviación estándar del pH",
    title="Estabilidad del pH con los productos",
    palette=PALETTE,
    save_html=f"{ROOT_IMAGEN}/ph_stability3.html"
)


[INFO] Gráfico guardado en /Users/juandavidrincon/Documents/galileo/images/segundo_ensayo_acidez_h2o/ph_stability3.html


In [32]:
stab

,n,pH_mean,pH_sd,pH_range,pH_cv_pct,ORP_mean,ORP_sd,ORP_range,ORP_cv_pct
producto,,,,,,,,,
ACIDO FOSFORICO,6,3.925,0.774,1.76,20.200,885.5,69.566,159,7.881
ACIDO ACETICO,9,4.210,0.795,2.15,18.999,824.0,69.521,186,8.486
CLAP,9,3.880,1.092,2.84,25.638,694.0,109.386,336,16.386
Citroquim Aqua,9,4.360,1.191,3.14,25.891,621.0,157.800,447,28.535
Selko pH,9,4.180,1.258,3.21,26.655,521.0,190.163,548,36.314


In [33]:
fig_rel = plot_orp_vs_ph(
    df_post,                      # tu DataFrame con pH y ORP
    #save_html=f"{ROOT_IMAGEN}/orp_ph.html"
)
fig_rel
s3.save_plotly_html(fig_rel, "orp_ph.html")

In [34]:
fig = plot_scatter_by_group(
    df=df_post,
    x_col="pH",
    y_col="ORP",
    color_col="producto",
    #annotation=(6.5, 650, "condición inicial"),
    title="Relación entre ORP y pH",
    palette=PALETTE,
    #save_html=f"{ROOT_IMAGEN}/relacion_orp_ph.html",

)
fig.show()
s3.save_plotly_html(fig, "relacion_orp_ph.html")

In [35]:
df_post

,producto,pH,ORP,FCL,TCL,dia,hora,ref,timestamp,elapsed_h
14,Citroquim Aqua,5.72,670,3.26,3.27,2025-07-22,12:00:00,0,2025-07-22 12:00:00,0.000000
15,Citroquim Aqua,4.37,755,3.19,3.53,2025-07-22,12:00:00,1,2025-07-22 12:00:00,0.000000
16,Citroquim Aqua,3.28,775,2.23,2.58,2025-07-22,12:00:00,2,2025-07-22 12:00:00,0.000000
17,Selko pH,5.68,722,4.29,7.36,2025-07-22,03:00:00,0,2025-07-22 03:00:00,0.000000
18,Selko pH,4.19,784,3.21,4.30,2025-07-22,03:00:00,1,2025-07-22 03:00:00,0.000000
19,Selko pH,3.52,400,1.22,2.87,2025-07-22,03:00:00,2,2025-07-22 03:00:00,0.000000
20,CLAP,5.62,674,4.02,5.25,2025-07-22,04:10:00,0,2025-07-22 04:10:00,0.000000
21,CLAP,4.26,717,3.67,4.34,2025-07-22,04:10:00,1,2025-07-22 04:10:00,0.000000
22,CLAP,3.48,775,3.37,4.26,2025-07-22,04:10:00,2,2025-07-22 04:10:00,0.000000
23,ACIDO ACETICO,5.25,703,3.56,4.31,2025-07-22,05:00:00,0,2025-07-22 05:00:00,0.000000


In [36]:
fig = plot_scatter_by_group(
    df=df_post,
    x_col="ORP",
    y_col="TCL",
    color_col="producto",
    #annotation=(6.5, 650, "condición inicial"),
    title="Relación entre TCL y ORP",
    palette=PALETTE,
    #save_html=f"{ROOT_IMAGEN}/relacion_tcl_orp.html",

)
fig.show()
s3.save_plotly_html(fig, "relacion_tcl_orp.html")

In [37]:
fig = plot_scatter_by_group(
    df=df_post,
    x_col="pH",
    y_col="TCL",
    color_col="producto",
    #annotation=(6.5, 650, "condición inicial"),
    title="Relación entre TCL y ORP",
    palette=PALETTE,
    #save_html=f"{ROOT_IMAGEN}/relacion_tcl_ph.html",

)
fig.show()
s3.save_plotly_html(fig, "relacion_tcl_ph.html")

In [38]:
df_post_ = df_post[df_post["elapsed_h"]>0]

In [39]:
stab = (
        df_post_.groupby(["producto", "ref"])
          .agg(
              n   = ("pH",  "count"),
              # ---- pH ----
              pH_mean   = ("pH",  "median"),
              pH_sd     = ("pH",  "std"),
              pH_range  = ("pH",  lambda x: x.max() - x.min()),
              pH_cv_pct = ("pH",  lambda x: x.std() / x.mean() * 100),

              # ---- ORP ----
              ORP_mean   = ("ORP", "median"),
              ORP_sd     = ("ORP", "std"),
              ORP_range  = ("ORP", lambda x: x.max() - x.min()),
              ORP_cv_pct = ("ORP", lambda x: x.std() / x.mean() * 100),
          )
          .round(3)
          .sort_values(["producto", "ref"]))
stab

n  pH_mean  pH_sd  pH_range  pH_cv_pct  ORP_mean  \
producto        ref                                                     
ACIDO ACETICO   0    3     4.97  0.151      0.29      3.014     725.0   
                1    3     4.21  0.235      0.45      5.473     824.0   
                2    3     3.16  0.261      0.50      8.040     892.0   
ACIDO FOSFORICO 0    3     4.44  0.172      0.32      3.814     829.0   
                1    3     3.04  0.272      0.51      8.642     951.0   
CLAP            0    3     5.68  0.257      0.50      4.571     689.0   
                1    3     3.88  0.244      0.46      6.136     737.0   
                2    3     3.10  0.263      0.50      8.228     550.0   
Citroquim Aqua  0    3     6.16  0.318      0.59      5.277     516.0   
                1    3     4.36  0.272      0.53      6.136     654.0   
                2    3     3.29  0.275      0.54      8.208     382.0   
Selko pH        0    3     6.32  0.265      0.53      4.191     647.0   
                1    3     4.18  0.236      0.45      5.542     521.0   
                2    3     3.48  0.253      0.48      7.087     246.0   

                      ORP_sd  ORP_range  ORP_cv_pct  
producto        ref                                  
ACIDO ACETICO   0     28.618         51       3.867  
                1     11.676         23       1.421  
                2     10.583         20       1.181  
ACIDO FOSFORICO 0     25.384         49       3.091  
                1     13.000         23       1.377  
CLAP            0      8.737         17       1.272  
                1     19.975         39       2.692  
                2    159.361        316      27.763  
Citroquim Aqua  0     90.886        181      17.290  
                1     45.574         85       6.772  
                2    231.656        443      50.323  
Selko pH        0     34.117         66       5.193  
                1    153.657        271      25.412  
                2    120.239        214      38.870

In [40]:
# Mostrar información básica del dataset
print("=== INFORMACIÓN BÁSICA DEL DATASET ===")
print(f"Dimensiones: {df_post_.shape}")
print(f"Columnas: {df_post_.columns.tolist()}")
print(f"Productos únicos: {df_post_['producto'].unique()}")
df_chlorine = df_post_.dropna(subset=['FCL', 'TCL']).copy()

=== INFORMACIÓN BÁSICA DEL DATASET ===
Dimensiones: (42, 10)
Columnas: ['producto', 'pH', 'ORP', 'FCL', 'TCL', 'dia', 'hora', 'ref', 'timestamp', 'elapsed_h']
Productos únicos: ['Citroquim Aqua' 'Selko pH' 'CLAP' 'ACIDO ACETICO' 'ACIDO FOSFORICO']


In [41]:
df_chlorine

,producto,pH,ORP,FCL,TCL,dia,hora,ref,timestamp,elapsed_h
42,Citroquim Aqua,5.66,516,0.37,0.69,2025-07-23,11:00:00,0,2025-07-23 11:00:00,23.000000
43,Citroquim Aqua,4.20,654,0.36,0.36,2025-07-23,11:00:00,1,2025-07-23 11:00:00,23.000000
44,Citroquim Aqua,3.11,382,0.31,0.71,2025-07-23,11:00:00,2,2025-07-23 11:00:00,23.000000
45,Selko pH,6.32,647,1.04,1.17,2025-07-23,11:00:00,0,2025-07-23 11:00:00,32.000000
46,Selko pH,4.08,521,0.16,1.43,2025-07-23,11:00:00,1,2025-07-23 11:00:00,32.000000
47,Selko pH,3.38,246,0.10,0.10,2025-07-23,11:00:00,2,2025-07-23 11:00:00,32.000000
48,CLAP,5.83,689,2.47,2.74,2025-07-23,11:00:00,0,2025-07-23 11:00:00,30.833333
49,CLAP,3.79,737,0.94,1.12,2025-07-23,11:00:00,1,2025-07-23 11:00:00,30.833333
50,CLAP,2.99,550,0.24,0.58,2025-07-23,11:00:00,2,2025-07-23 11:00:00,30.833333
51,ACIDO ACETICO,4.90,773,1.83,2.32,2025-07-23,11:00:00,0,2025-07-23 11:00:00,30.000000


In [42]:


def calculate_consistency_metrics(group):
    """Calcula métricas de consistencia para un grupo de datos"""
    fcl_values = group['FCL'].values
    orp_values = group['ORP'].values
    ph_values = group['pH'].values

    # Estadísticas básicas FCL
    fcl_mean = np.mean(fcl_values)
    fcl_std = np.std(fcl_values, ddof=1)  # Desviación estándar muestral
    fcl_cv = (fcl_std / fcl_mean) * 100 if fcl_mean != 0 else 0  # Coeficiente de variación
    fcl_min = np.min(fcl_values)
    fcl_max = np.max(fcl_values)
    fcl_range = fcl_max - fcl_min

    ph_mean = np.mean(ph_values)
    ph_std = np.std(ph_values, ddof=1)
    ph_cv = (ph_std / ph_mean) * 100
    ph_min = np.min(ph_values)
    ph_max = np.max(ph_values)
    ph_range = ph_max - ph_min


    # Estadísticas básicas ORP
    orp_mean = np.mean(orp_values)
    orp_std = np.std(orp_values, ddof=1)
    orp_cv = (orp_std / orp_mean) * 100 if orp_mean != 0 else 0

    return pd.Series({
        'n_samples': len(fcl_values),
        'FCL_mean': fcl_mean,
        'FCL_std': fcl_std,
        'FCL_cv': fcl_cv,
        'FCL_min': fcl_min,
        'FCL_max': fcl_max,
        'FCL_range': fcl_range,
        'ORP_mean': orp_mean,
        'ORP_std': orp_std,
        'ORP_cv': orp_cv,
        'ph_mean': ph_mean,
        'ph_std': ph_std,
        'ph_cv': ph_cv,
        'ph_min': ph_min,
        'ph_max': ph_max,
        'ph_range': ph_range,

    })

consistency_stats = df_chlorine.groupby('producto').apply(calculate_consistency_metrics)
consistency_stats.sort_values('FCL_cv')

,n_samples,FCL_mean,FCL_std,FCL_cv,FCL_min,FCL_max,FCL_range,ORP_mean,ORP_std,ORP_cv,ph_mean,ph_std,ph_cv,ph_min,ph_max,ph_range
producto,,,,,,,,,,,,,,,,
ACIDO FOSFORICO,2.0,2.705000,0.219203,8.103627,2.55,2.86,0.31,890.500000,86.974134,9.766888,3.695000,1.053589,28.513914,2.95,4.44,1.49
Citroquim Aqua,3.0,0.346667,0.032146,9.272741,0.31,0.37,0.06,517.333333,136.004902,26.289607,4.323333,1.279466,29.594434,3.11,5.66,2.55
ACIDO ACETICO,3.0,1.523333,0.957566,62.859936,0.45,2.29,1.84,837.666667,67.678160,8.079366,4.013333,0.933024,23.248100,3.04,4.90,1.86
CLAP,3.0,1.216667,1.140453,93.735873,0.24,2.47,2.23,658.666667,97.120200,14.744970,4.203333,1.464423,34.839553,2.99,5.83,2.84
Selko pH,3.0,0.433333,0.526245,121.441052,0.10,1.04,0.94,471.333333,205.061779,43.506742,4.593333,1.535752,33.434362,3.38,6.32,2.94


In [44]:
f = plot_stability_bar(
    consistency_stats,
    y_col="FCL_cv",
    y_label="Coeficiente de Variación (CV)",
    title="Estabilidad del FCL por productos",
    palette=PALETTE,
    #save_html=f"{ROOT_IMAGEN}fcl_stability.html"
)
s3.save_plotly_html(f, f"fcl_stability.html")

In [45]:
consistency_stats

,n_samples,FCL_mean,FCL_std,FCL_cv,FCL_min,FCL_max,FCL_range,ORP_mean,ORP_std,ORP_cv,ph_mean,ph_std,ph_cv,ph_min,ph_max,ph_range
producto,,,,,,,,,,,,,,,,
ACIDO ACETICO,3.0,1.523333,0.957566,62.859936,0.45,2.29,1.84,837.666667,67.678160,8.079366,4.013333,0.933024,23.248100,3.04,4.90,1.86
ACIDO FOSFORICO,2.0,2.705000,0.219203,8.103627,2.55,2.86,0.31,890.500000,86.974134,9.766888,3.695000,1.053589,28.513914,2.95,4.44,1.49
CLAP,3.0,1.216667,1.140453,93.735873,0.24,2.47,2.23,658.666667,97.120200,14.744970,4.203333,1.464423,34.839553,2.99,5.83,2.84
Citroquim Aqua,3.0,0.346667,0.032146,9.272741,0.31,0.37,0.06,517.333333,136.004902,26.289607,4.323333,1.279466,29.594434,3.11,5.66,2.55
Selko pH,3.0,0.433333,0.526245,121.441052,0.10,1.04,0.94,471.333333,205.061779,43.506742,4.593333,1.535752,33.434362,3.38,6.32,2.94


In [46]:
plot_stability_bar(
    consistency_stats,
    y_col="ORP_std",
    y_label="Fluctuaciones (std)",
    title="Fluctuaciones del ORP por productos",
    palette=PALETTE,
    #save_html=f"{ROOT_IMAGEN}/orp_stability.html"
)

In [47]:
plot_stability_bar(
    consistency_stats,
    y_col="ph_std",
    y_label="Fluctuaciones (std)",
    title="Fluctuaciones del pH por productos",
    palette=PALETTE,
    #save_html=f"{ROOT_IMAGEN}/ph_stability.html"
)

In [48]:
consistency_stats

,n_samples,FCL_mean,FCL_std,FCL_cv,FCL_min,FCL_max,FCL_range,ORP_mean,ORP_std,ORP_cv,ph_mean,ph_std,ph_cv,ph_min,ph_max,ph_range
producto,,,,,,,,,,,,,,,,
ACIDO ACETICO,3.0,1.523333,0.957566,62.859936,0.45,2.29,1.84,837.666667,67.678160,8.079366,4.013333,0.933024,23.248100,3.04,4.90,1.86
ACIDO FOSFORICO,2.0,2.705000,0.219203,8.103627,2.55,2.86,0.31,890.500000,86.974134,9.766888,3.695000,1.053589,28.513914,2.95,4.44,1.49
CLAP,3.0,1.216667,1.140453,93.735873,0.24,2.47,2.23,658.666667,97.120200,14.744970,4.203333,1.464423,34.839553,2.99,5.83,2.84
Citroquim Aqua,3.0,0.346667,0.032146,9.272741,0.31,0.37,0.06,517.333333,136.004902,26.289607,4.323333,1.279466,29.594434,3.11,5.66,2.55
Selko pH,3.0,0.433333,0.526245,121.441052,0.10,1.04,0.94,471.333333,205.061779,43.506742,4.593333,1.535752,33.434362,3.38,6.32,2.94


In [49]:
plot_stability_bar(
    consistency_stats,
    y_col="FCL_std",
    y_label="Fluctuaciones (std)",
    title="Fluctuaciones del TCL por productos",
    palette=PALETTE,
    #save_html=f"{ROOT_IMAGEN}/fcl_stability.html"
)

In [50]:


def create_scatter_plot(
    df: pd.DataFrame,
    x_col: str,
    y_col: str,
    color_col: str = None,
    size_col: str = None,
    hover_data: list[str] = None,
    title: str = "Scatter Plot",
    save_html: str = None,
) -> px.scatter:
    """
    Crea un scatter plot interactivo con Plotly Express.

    Parámetros:
    - df: DataFrame de Pandas con los datos.
    - x_col: Nombre de la columna para el eje X.
    - y_col: Nombre de la columna para el eje Y.
    - color_col: (opcional) Columna para colorear los puntos.
    - size_col: (opcional) Columna para escalar el tamaño de los puntos.
    - hover_data: (opcional) Lista de columnas adicionales que se muestran al pasar el cursor.
    - title: Título del gráfico.
    - save_html: (opcional) Ruta para guardar el gráfico como archivo HTML.

    Retorna:
    - fig: Objeto Plotly Figure.
    """
    fig = px.scatter(
        df,
        x=x_col,
        y=y_col,
        color=color_col,
        size=size_col,
        hover_data=hover_data,
        title=title,
    )

    fig.update_layout(
        title=dict(x=0.5),
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(color='black'),
    )

    if save_html:
        fig.write_html(save_html)

    return fig


In [51]:
fig = create_scatter_plot(
    df=consistency_stats.reset_index(),
    x_col="FCL_mean",
    y_col="FCL_std",
    color_col="producto",
    hover_data=["producto"],
    title="Cloro Libre vs Cloro Total"
)
fig.show()

In [164]:
fig = create_scatter_plot(
    df=consistency_stats.reset_index(),
    x_col="FCL_mean",
    y_col="ORP_mean",
    color_col="producto",
    hover_data=["producto"],
    title="Cloro Libre vs Cloro Total"
)
fig.show()

In [52]:
fig = create_scatter_plot(
    df=consistency_stats.reset_index(),
    x_col="FCL_mean",
    y_col="ph_mean",
    color_col="producto",
    hover_data=["producto"],
    title="Cloro Libre vs Cloro Total"
)
fig.show()